# Working with Data - Computer Lab for Guest Lecture Julia Lane

In this computer lab we will learn more more details and practice data work to enhance the content of the lecture presented by Julia Lane on responsible data use. We will address a research question, think about data and measurement errors, and manipulate data. 

OUTLINE: 
1. Define a research question 
2. Think about what data are available 
3. Think about possible measurement errors 
4. Think about the interpretation of your results 
5. Inform your results by linking datasets 

# 1. Define a reserach question
Which Community Districts in NYC show the highest number of complaints?

# 2. Think about what data are available
Find suitable data by searching the CUSP Data Catalog https://datahub.cusp.nyu.edu/catalog. You can use Urban Profiler to investigate the Metadata associated with each dataset. Using this tool will help you to decide which attributes of the data you need to answer your question so you don't have to load the entire dataset. 

In [2]:
import os
import pandas as pd
import numpy as np
import re
PUIdata = os.getenv('PUIDATA')

In [69]:
# reading in data, only columns that interest us
data = pd.read_csv('https://data.cityofnewyork.us/resource/erm2-nwe9.csv', usecols=[0,1,8,23])

# 3. Think about possible measurement errors
Do you see any problems regarding possible measurement error? Think about who is represented in the data, ommissions, duplications, content error, missing data, etc. 

In [71]:
# counting unique community districts
print "Number of unique Community Districts: %d" %len(data.groupby(['Community Board'])['Community Board'].nunique())
print "We should have 59 Community Districts."
print "We have %d too many Community Districts in our dataset." %(len(data.groupby
                                                                     (['Community Board'])['Community Board'].nunique()) - 59)

Number of unique Community Districts: 61
We should have 59 Community Districts.
We have 2 too many Community Districts in our dataset.


In [72]:
# inspecting the data
print data.groupby(['Community Board']).count().head(1)
print data.groupby(['Community Board']).count().tail(1)

                 Unique Key  Created Date  Incident Zip
Community Board                                        
0 Unspecified             8             8             0
                      Unique Key  Created Date  Incident Zip
Community Board                                             
Unspecified BROOKLYN           1             1             0


Looking into our dataset we have 8 unique values with an Unspecified Community District and 1 unique value with an Unspecified Brooklyn Community District. This explains where the two extra Community Districts come from.

In [73]:
# dropping unspecified community boards
data = data[data['Community Board'] != "0 Unspecified"]
data = data[data['Community Board'] != "Unspecified BROOKLYN"]

In [236]:
# reducing data to Community District Level and listing the 10 Community Districts with the most complains
dataframe1 = pd.DataFrame(data.groupby(['Community Board'])['Unique Key'].count(), )
dataframe1.rename(columns={'Unique Key' : 'no_complaints'}, inplace=True)
dataframe1.sort(columns='no_complaints', ascending=False).head(10)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


no_complaints
Community Board               
12 MANHATTAN                70
03 MANHATTAN                48
05 QUEENS                   36
04 BROOKLYN                 34
01 BROOKLYN                 32
17 BROOKLYN                 32
01 QUEENS                   32
11 BRONX                    29
07 BRONX                    28
09 QUEENS                   27

# 4. Think about the interpretation of your results?
What do you have to keep in mind when interpreting your results? Are they generable? Does the way the data is collected influence your results? To better inform city agancies it might be good to explore in more detail the underlying dempgraphics/infrastructure of a Community District becasue this might influence 311 calls. You can do this by merging external data on the Community District level to your analysis data. 

In [109]:
os.system("curl -O http://cosmo.nyu.edu/~fb55/PUI2016/data/Final_Demographics.csv")
os.system("mv Final_Demographics.csv " +'/'+ PUIdata)

0

In [110]:
# Population by Community District
df_pop = pd.read_csv(PUIdata + "/Final_Demographics.csv")

In [124]:
# How many community districts are in file? 
print "Number of community districts: %d" %len(df_pop.groupby(['cd_id'])['cd_id'].nunique())

Number of community districts: 59


In [220]:
# Manipulating data to get some information on demographics by Community District. 
dataframe2 = df_pop[['FIPS', 'cd_id', 'Total Population', 
                          '% Population 5 Years And Over: Spanish or Spanish Creole: Speak English Less Than "very Well"', 
                         '% Population 25 Years and over: Less Than High School', 
                         'Median household income (In 2014 Inflation Adjusted Dollars)']]
dataframe2.head()

FIPS cd_id  Total Population  \
0  3603701  BX08            106737   
1  3603702  BX12            134644   
2  3603703  BX10            121209   
3  3603704  BX11            135839   
4  3603705  BX03            172247   

   % Population 5 Years And Over: Spanish or Spanish Creole: Speak English Less Than "very Well"  \
0                                              15.24                                               
1                                               6.07                                               
2                                               6.92                                               
3                                              13.66                                               
4                                              24.14                                               

   % Population 25 Years and over: Less Than High School  \
0                                              16.93       
1                                              19.78       
2                                              17.34       
3                                              22.84       
4                                              38.02       

   Median household income (In 2014 Inflation Adjusted Dollars)  
0                                              54224             
1                                              44906             
2                                              54962             
3                                              47910             
4                                              22041

In [139]:
os.system("curl -O http://cosmo.nyu.edu/~fb55/PUI2016/data/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")
os.system("mv ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv " +'/'+ PUIdata)

0

In [140]:
# Infrastructure by Community District
df_infr = pd.read_csv(PUIdata + "/ACS_Computer_Use_and_Internet_2014_1Year_Estimate.csv")

In [144]:
# Check variables in file
df_infr.columns
#df_infr.head()

Index([u'FIPS', u'Geographic Identifier', u'Qualifying Name', u'Households',
       u'Households: With An Internet Subscription',
       u'Households: Dial-Up Alone', u'Households: Dsl',
       u'Households: With Mobile Broadband',
       u'Households: Without Mobile Broadband', u'Households: Cable Modem',
       u'Households: With Mobile Broadband.1',
       u'Households: Without Mobile Broadband.1', u'Households: Fiber-Optic',
       u'Households: With Mobile Broadband.2',
       u'Households: Without Mobile Broadband.2',
       u'Households: Satellite Internet Service',
       u'Households: With Mobile Broadband.3',
       u'Households: Without Mobile Broadband.3',
       u'Households: Two or More Fixed Broadband Types, or Other',
       u'Households: With Mobile Broadband.4',
       u'Households: Without Mobile Broadband.4',
       u'Households: Mobile Broadband Alone or With Dialup',
       u'Households: Internet Access Without A Subscription',
       u'Households: No Internet Acc

In [164]:
# How many community districts are in file? 
print "Number of community districts: %d" %len(df_infr.groupby(['Qualifying Name'])['Qualifying Name'].nunique())
print "We are missing %d districts." %(59-len(df_infr.groupby(['Qualifying Name'])['Qualifying Name'].nunique()))

Number of community districts: 55
We are missing 4 districts.


In [168]:
# taking a closer look at our data
df_infr.groupby(['Qualifying Name']).count().sort(columns="FIPS", ascending=False).head()

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


FIPS  \
Qualifying Name                                            
NYC-Bronx Community District 1 & 2--Hunts Point...     2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...     2   
NYC-Manhattan Community District 4 & 5--Chelsea...     2   
NYC-Manhattan Community District 1 & 2--Battery...     2   
NYC-Queens Community District 12--Jamaica, New ...     1   

                                                    Geographic Identifier  \
Qualifying Name                                                             
NYC-Bronx Community District 1 & 2--Hunts Point...                      2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...                      2   
NYC-Manhattan Community District 4 & 5--Chelsea...                      2   
NYC-Manhattan Community District 1 & 2--Battery...                      2   
NYC-Queens Community District 12--Jamaica, New ...                      1   

                                                    Households  \
Qualifying Name                                                  
NYC-Bronx Community District 1 & 2--Hunts Point...           2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...           2   
NYC-Manhattan Community District 4 & 5--Chelsea...           2   
NYC-Manhattan Community District 1 & 2--Battery...           2   
NYC-Queens Community District 12--Jamaica, New ...           1   

                                                    Households: With An Internet Subscription  \
Qualifying Name                                                                                 
NYC-Bronx Community District 1 & 2--Hunts Point...                                          2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...                                          2   
NYC-Manhattan Community District 4 & 5--Chelsea...                                          2   
NYC-Manhattan Community District 1 & 2--Battery...                                          2   
NYC-Queens Community District 12--Jamaica, New ...                                          1   

                                                    Households: Dial-Up Alone  \
Qualifying Name                                                                 
NYC-Bronx Community District 1 & 2--Hunts Point...                          2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...                          2   
NYC-Manhattan Community District 4 & 5--Chelsea...                          2   
NYC-Manhattan Community District 1 & 2--Battery...                          2   
NYC-Queens Community District 12--Jamaica, New ...                          1   

                                                    Households: Dsl  \
Qualifying Name                                                       
NYC-Bronx Community District 1 & 2--Hunts Point...                2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...                2   
NYC-Manhattan Community District 4 & 5--Chelsea...                2   
NYC-Manhattan Community District 1 & 2--Battery...                2   
NYC-Queens Community District 12--Jamaica, New ...                1   

                                                    Households: With Mobile Broadband  \
Qualifying Name                                                                         
NYC-Bronx Community District 1 & 2--Hunts Point...                                  2   
NYC-Bronx Community District 3 & 6--Belmont, Ne...                                  2   
NYC-Manhattan Community District 4 & 5--Chelsea...                                  2   
NYC-Manhattan Community District 1 & 2--Battery...                                  2   
NYC-Queens Community District 12--Jamaica, New ...                                  1   

                                                    Households: Without Mobile Broadband  \
Qualifying Name                                                                            
NYC-Bronx Community District 1 & 2--Hunts Point...                        

Eight community districts seem to be bundled into four which would explain the four missing districts from earlier

In [217]:
dataframe3 = df_infr[['Households', 'Qualifying Name', '% Households: With An Internet Subscription']]
dataframe3.head(3)

Households                                    Qualifying Name  \
0       42035  NYC-Bronx Community District 8--Riverdale, New...   
1       44830  NYC-Bronx Community District 12--Wakefield, Ne...   
2       47050  NYC-Bronx Community District 10--Co-op City, N...   

   % Households: With An Internet Subscription  
0                                        75.64  
1                                        71.92  
2                                        69.56

In [218]:
# Aggregating 
dataframe3['% Households: With An Internet Subscription'] = (
    dataframe2['% Households: With An Internet Subscription']/100) 
dataframe3['% Households: Dial-Up Alone'] = (df_infr['Households: Dial-Up Alone'] / df_infr['Households'])
dataframe3['% Households: With Dial-Up Internet Subscription Alone'] = (
    df_infr['Households: With Dial-Up Internet Subscription Alone']/df_infr['Households'])
dataframe3['% Households: With A Broadband Internet Subscription'] = (
    df_infr['Households: With A Broadband Internet Subscription'] / df_infr['Households'])
dataframe3['% Households: Dsl'] = (df_infr['Households: Dsl'] / df_infr['Households'])
dataframe3['% Households: Internet Access Without A Subscription'] = (
    df_infr['Households: Internet Access Without A Subscription'] / df_infr['Households'])

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [219]:
dataframe3.head(3)

Households                                    Qualifying Name  \
0       42035  NYC-Bronx Community District 8--Riverdale, New...   
1       44830  NYC-Bronx Community District 12--Wakefield, Ne...   
2       47050  NYC-Bronx Community District 10--Co-op City, N...   

   % Households: With An Internet Subscription  % Households: Dial-Up Alone  \
0                                     0.007564                     0.010563   
1                                     0.007192                     0.003971   
2                                     0.006956                     0.003358   

   % Households: With Dial-Up Internet Subscription Alone  \
0                                           0.009611        
1                                           0.003971        
2                                           0.003358        

   % Households: With A Broadband Internet Subscription  % Households: Dsl  \
0                                           0.736125              0.066920   
1                                           0.701205              0.018938   
2                                           0.687205              0.039596   

   % Households: Internet Access Without A Subscription  
0                                           0.050410     
1                                           0.042182     
2                                           0.061254

# 5. Inform your results by linking datasets
Now you want to link the three data frames to produce summary statistics for Community Districts which show a high number of complaints vs. Community Districts which show a lower number of complaints. Please keep in mind that the identifiers used for the linkage (Community Dostrict IDs) should be recored the same way. Use regular expressions to harmonize the identifiers if possible. The identifiers should look like BK01, BK02, etc.
https://docs.python.org/2/library/re.html

In [244]:
# Harmonize identifier of dataframe 1
dataframe1.head()

no_complaints
Community Board                
01 BRONX                     25
01 BROOKLYN                  32
01 MANHATTAN                  5
01 QUEENS                    32
01 STATEN ISLAND              8

In [245]:
dataframe1.columns

Index([u'no_complaints'], dtype='object')

In [ ]:
# My dataframe is not aligned correctly

In [ ]:
# Harmonize identifier of dataframe 2

In [ ]:
# Harmonize identifier of dataframe 3

In [ ]:
# Link the 3 dataframes

In [ ]:
# Are the demographics and infrastructure different in Community Districts that show more complaints than others?